todoリストのjsonファイルから抜き出す  
メール本文確認用  
main.js最終行の"console.log('TODOs file path:', TODOS_FILE);"で出力されるパスにjsonファイルが格納されている  

In [ ]:
import json
import os
import datetime
from dotenv import load_dotenv

today = datetime.date.today()
formatted_date = today.strftime("%Y%m%d")

load_dotenv(".env")
TODOS_JSON_PATH = os.getenv("TODOS_JSON_PATH")

with open(TODOS_JSON_PATH, "r", encoding="utf-8") as f:
    json_data = json.load(f)

done_true = [todo for todo in json_data if todo["done"] == True]
done_false = [todo for todo in json_data if todo["done"] == False]

def format_todos(todos):
    return "\n\n".join([f"{todo['title']}\nメモ:\n{todo['description']}" for todo in todos])

done_true_formatted = format_todos(done_true)
done_false_formatted = format_todos(done_false)

mail_body = f"完了:\n{done_true_formatted}\n\n実施中:\n{done_false_formatted}"

with open("../nippou/"+formatted_date+".txt", "w", encoding="utf-8") as f:
    f.write(mail_body)

メール送信確認用

In [ ]:
import smtplib
from email.mime.text import MIMEText

GMAIL_ADDRESS = os.getenv("GMAIL_ADDRESS")
GMAIL_PASSWORD = os.getenv("GMAIL_PASSWORD")

msg = MIMEText(mail_body)
msg['Subject'] = '本日の進捗について('+formatted_date+')'
msg['From'] = ""
msg['To'] = ''

smtp_server = 'smtp-mail.outlook.com'
smtp_port = 465
username = GMAIL_ADDRESS
password = GMAIL_PASSWORD

with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
    server.login(username, password)
    server.send_message(msg)

print('メールが送信されました')


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

offic365_address = os.getenv("offic365_address")
offic365_password = os.getenv("offic365_password")

# メール設定
smtp_server = "smtp.office365.com"
smtp_port = 587
sender_email = offic365_address
sender_password = offic365_password
recipient_email = offic365_address

# メッセージの作成
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = recipient_email
message["Subject"] = "テストメール from Python"

# メール本文
body = "これはPythonから送信されたテストメールです。"
message.attach(MIMEText(body, "plain"))

try:
    # SMTPサーバーへの接続
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.set_debuglevel(1)
        server.ehlo()  # サーバーとの対話を開始
        server.starttls()  # TLS暗号化を開始
        server.ehlo()  # 再度対話を開始（暗号化後）
        
        # 明示的な認証方法の指定
        server.login(sender_email, sender_password)
        
        # メールの送信
        server.send_message(message)
        print("メールが正常に送信されました。")
except Exception as e:
    print(f"エラーが発生しました: {e}")